In [1]:
DATA_PATH = "../data/raw/survey_results_public.csv"
EXPORT_PATH = "../data/processed/1_preprocessed_df.pkl"

REPLACE_DICT = {
    'YearsCodePro' : {"Less than 1 year" : 0, "More than 50 years" : 51},
    'YearsCode' : {"Less than 1 year" : 0, "More than 50 years" : 51}
}

In [2]:
import pandas as pd
import numpy as np
import logging
import pickle

# Functions

## Split Answers

In [3]:
raw_df = pd.read_csv(DATA_PATH)
ans_df = raw_df.copy()

In [4]:
# Split the answers
ans_df["LanguageHaveWorkedWith"][0]
# check if col contains ;
ans_df['LanguageHaveWorkedWith'].str.contains(';')
# split the answer based on the ; and put it on list
ans_df['LanguageHaveWorkedWith'].str.split(';')

0        [C++, HTML/CSS, JavaScript, Objective-C, PHP, ...
1                                     [JavaScript, Python]
2                           [Assembly, C, Python, R, Rust]
3                                 [JavaScript, TypeScript]
4                      [Bash/Shell, HTML/CSS, Python, SQL]
                               ...                        
83434                               [Clojure, Kotlin, SQL]
83435                                                  NaN
83436                               [Groovy, Java, Python]
83437            [Bash/Shell, JavaScript, Node.js, Python]
83438         [Delphi, Elixir, HTML/CSS, Java, JavaScript]
Name: LanguageHaveWorkedWith, Length: 83439, dtype: object

In [5]:
def split_answers(data_series, delimiter=";"):
    """
    Split multiple answers in a single string
    to a list od single strings each contains single answers
    
    Parameters:
    * data_series (pd.Series) : string series with answers
    * delimiter (string) : string to split with default ";"
    """
    
    def is_splittable(pd_series, delimiter):
        return pd_series.str.contains(';')
    
    def split_answer(pd_series, delimiter):
        return pd_series.str.split(delimiter)
    
    # Check if there any splittable answers if not return the data series
    splittable_values = is_splittable(data_series, delimiter)
    if not splittable_values.any():
        return data_series
    
    # Split values to list
    modified_series = split_answer(data_series, delimiter)
    
    # Replace NAs with empty lists
    mask_null = modified_series.isnull()
    modified_series.loc[mask_null] = modified_series.loc[mask_null].apply(lambda x: [])
    
    return modified_series

In [6]:
object_cols = ans_df.select_dtypes(include='object').columns.tolist()

for col in object_cols:
    ans_df[col] = split_answers(ans_df[col])

## Replace Answers

In [11]:
for col, replacement in REPLACE_DICT.items():
    ans_df[col] =  ans_df[col].replace(replacement).astype(np.float32)

## Visually verify results

In [13]:
ans_df["YearsCode"].unique()

array([nan,  7., 17.,  3.,  4.,  6., 16., 12., 15., 10., 40.,  9., 26.,
       14., 39., 20.,  8., 19.,  5.,  0., 22.,  2.,  1., 34., 21., 13.,
       25., 24., 30., 31., 18., 38., 51., 27., 41., 42., 35., 23., 28.,
       11., 37., 44., 43., 36., 33., 45., 29., 50., 46., 32., 47., 49.,
       48.], dtype=float32)

In [14]:
ans_df["YearsCodePro"].unique()

array([nan, 10.,  4.,  5.,  6.,  2., 30.,  9., 18., 12., 21.,  1., 16.,
        0., 15.,  3., 35.,  7.,  8., 17., 14., 26., 25., 20., 50., 34.,
       11., 24., 22., 13., 31., 23., 39., 41., 27., 28., 19., 33., 51.,
       37., 29., 32., 43., 40., 38., 45., 42., 46., 36., 44., 47., 48.,
       49.], dtype=float32)

In [18]:
i = ans_df.sample(1).index[0]
print(raw_df["LanguageHaveWorkedWith"].iloc[i])
print(ans_df["LanguageHaveWorkedWith"].iloc[i])

C#;JavaScript;Perl;Rust;TypeScript
['C#', 'JavaScript', 'Perl', 'Rust', 'TypeScript']


In [19]:
i = ans_df.sample(1).index[0]
print(raw_df["DevType"].iloc[i])
print(ans_df["DevType"].iloc[i])

Developer, QA or test
['Developer, QA or test']


## Export Data

In [20]:
ans_df.to_pickle(EXPORT_PATH)